In [1]:
from imprint.nb_util import setup_nb

# setup_nb is a handy function for setting up some nice plotting defaults.
setup_nb()
import scipy.stats
import matplotlib.pyplot as plt

import imprint as ip
import numpy as np
import jax


In [4]:
grid = ip.cartesian_grid(
        [-3, -3], [3, 3], n=[20, 20], null_hypos=[ip.hypo("theta1 < theta0")]
    )
lam = 0.025
K = 100
n = 50

NameError: name 'x' is not defined

NameError: name 'BinomialClosed' is not defined

In [13]:
class BinomialClosed:
    def __init__(self, seed, max_K, *, n):
        self.family = "binomial"
        self.family_params = {"n": n[0]}
        self.n=n
        key = jax.random.PRNGKey(seed)
        splitkeys = jax.random.split(key, num=2)
        self.samples_arm1 = jax.random.uniform(
            splitkeys[1], shape=(max_K, n[1]), dtype=np.float32
        )
        
        self.samples_arm0 = jax.random.uniform(
            splitkeys[0], shape=(max_K, n[0]), dtype=np.float32
        )

    def sim_batch(self, begin_sim, end_sim, theta, null_truth, detailed=False):
        p = jax.scipy.special.expit(theta)
        # successes is a 2-dimensional array of shape: (n_tiles, n_sims = K)
        successes1 = np.sum(self.samples_arm1[None, begin_sim:end_sim, :] < p[:, None, None, 1],axis =2)
        successes0 = np.sum(self.samples_arm0[None, begin_sim:end_sim, :] < p[:, None, None, 0],axis =2)
        phat1 = successes1 / self.n[1]
        phat0 = successes0 / self.n[0]
        pooledphat = (phat1*self.n[1] + phat0*self.n[0]) / (self.n[1] + self.n[0])
        zstats = (phat1 - phat0) / np.sqrt(pooledphat * (1 - pooledphat) * (1 / self.n[0] + 1 / self.n[1]))
        #output is a 2-dimensional array of shape (n_tiles, n_sims = K)
        return -zstats


In [14]:
ip.validate(BinomialClosed, grid, lam, K=K, model_kwargs=dict(n=[100,100]))

,tie_sum,tie_est,tie_cp_bound,tie_bound
0,50,0.50,0.619283,0.897643
1,25,0.25,0.364503,0.689318
2,8,0.08,0.165946,0.435788
3,1,0.01,0.064543,0.242026
4,0,0.00,0.045007,0.202271
...,...,...,...,...
225,54,0.54,0.657288,0.994163
226,59,0.59,0.703765,0.994046
227,51,0.51,0.628852,0.953501
228,49,0.49,0.609668,0.918437
